In [6]:
import TransferLearn

In [4]:
conda install -c anaconda tensorflow

\ 
Note: you may need to restart the kernel to use updated packages.


In [8]:
df = TransferLearn.preprocess.read('sampleData/GDB17_shuf_large.txt')

In [9]:
charset = {'!', 'E', '#', '(', ')', '/', '\\', '%', '+', '@', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '=', 'B', 'Ge', 'Si', 'Se', 'Te', 'Br', 'C', 'Cl', 'F', 'H', 'I', 'N', 'O', 'S', '[', ']','c', 'n', 'o', 'P', 's','e'}
embed = max([len(smile) for smile in df.SMILES]) + 10
from sklearn.model_selection import train_test_split
smiles_train, smiles_test = train_test_split(df["SMILES"], test_size=0.01, random_state=42)

In [11]:
X_train, Y_train, char_to_int, int_to_char = TransferLearn.preprocess.OneHotVectorize(smiles_train,embed,charset)
X_test, Y_test, _,_ = TransferLearn.preprocess.OneHotVectorize(smiles_test,embed,charset)

In [12]:
model = TransferLearn.process.RNNTransferLearn.build_model(3, [100,100,100], [0.1,0.1,0.1], (X_train.shape[1],X_train.shape[2]), Y_train.shape[2])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 100)         58000     
_________________________________________________________________
dropout (Dropout)            (None, None, 100)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 100)         80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 100)         80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
dense (Dense)                (None, None, 44)          4

In [16]:
tfmodel = TransferLearn.process.RNNTransferLearn.tlmodel(model, 1, [100], [0.1], (X_train.shape[1],X_train.shape[2]), Y_train.shape[2], freeze_layer = True, fine_tune = False)


In [17]:
tfmodel.summary()
print("weights:", len(tfmodel.weights))
print("trainable_weights:", len(tfmodel.trainable_weights))
print("non_trainable_weights:", len(tfmodel.non_trainable_weights))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 100)         58000     
_________________________________________________________________
dropout (Dropout)            (None, None, 100)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 100)         80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 100)         80400     
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, None, 100)        

In [19]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

#response = pd.read_csv('Response.csv')
bounds = np.array([[10, 512],[10,512],[0.2,0.9],[0.2,0.9]]) 
kernel = RBF((16,16,0.1,0.1), ((10,512),(10,512),(0.2,0.9),(0.2,0.9)))

In [21]:
X = pd.DataFrame(response, columns = ['LSTM1','LSTM2','DP1','DP2'])
X = X.to_numpy()
y = pd.DataFrame(response, columns = ['loss'])
y = y.to_numpy()

NameError: name 'response' is not defined

In [20]:
gpmodel = TransferLearn.process.optimize.gpInit(kernel, n_restarts_optimizer=10000)
best = TransferLearn.process.optimize.gpOpt(gpmodel, X, y, X_train, Y_train, X_test, Y_test, bounds)

NameError: name 'X' is not defined